<a href="https://colab.research.google.com/github/ke-fan/ke-fan.github.io/blob/master/examples/ml/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/ke-fan/ml-kit/blob/main/examples/ml/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task
1. MLP demo (done)
2. RNN demo
3. LSTM demo
4. Transformer demo
5. GNN demo
6. Finetune

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Vision

In [ ]:
# 获取MNIST数据集，数字识别
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# 通过dataloader获取数据：批量，shuffle等支持
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X[N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### MLP

In [ ]:
# 多层感知机，多层全连接网络+非线性激活函数
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

# 定义模型网络结构
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# 损失函数
loss_fn = nn.CrossEntropyLoss()
# 求解最优化算法，与模型参数关联
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
# 定义模型训练与测试方法
def train(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        # 计算loss信息，并与模型关联（推测，关联后用于后续梯度计算和模型参数优化）
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        # 后向传播计算梯度
        loss.backward()
        # 通过梯度更新模型参数
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


def test(dataloader: DataLoader, model: nn.Module, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg loss: {test_loss:>8f}\n')

In [ ]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n{"-"*20}')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
--------------------
loss: 2.311415 [    0/60000]
loss: 2.299020 [ 6400/60000]
loss: 2.279966 [12800/60000]
loss: 2.271250 [19200/60000]
loss: 2.258297 [25600/60000]
loss: 2.228195 [32000/60000]
loss: 2.234925 [38400/60000]
loss: 2.199722 [44800/60000]
loss: 2.195564 [51200/60000]
loss: 2.167549 [57600/60000]
Test Error: 
 Accuracy: 40.9, Avg loss: 2.164597

Epoch 2
--------------------
loss: 2.176257 [    0/60000]
loss: 2.167387 [ 6400/60000]
loss: 2.111066 [12800/60000]
loss: 2.124904 [19200/60000]
loss: 2.075110 [25600/60000]
loss: 2.020886 [32000/60000]
loss: 2.046586 [38400/60000]
loss: 1.966213 [44800/60000]
loss: 1.963673 [51200/60000]
loss: 1.898258 [57600/60000]
Test Error: 
 Accuracy: 57.4, Avg loss: 1.903650

Epoch 3
--------------------
loss: 1.932389 [    0/60000]
loss: 1.908970 [ 6400/60000]
loss: 1.789696 [12800/60000]
loss: 1.831100 [19200/60000]
loss: 1.716206 [25600/60000]
loss: 1.676327 [32000/60000]
loss: 1.692014 [38400/60000]
loss: 1.586477 [44800/60000]
l

In [ ]:
import os

# 模型存储与加载1：存储模型状态
os.makedirs('tmp/pytorch', exist_ok=True)
torch.save(model.state_dict(), 'tmp/pytorch/model.pth')
print('Saved PyTorch Model State to tmp/pytorch/model.pth')
model = NeuralNetwork()
model.load_state_dict(torch.load('tmp/pytorch/model.pth'))

Saved PyTorch Model State to tmp/pytorch/model.pth


<All keys matched successfully>

In [ ]:
# 模型存储与加载2：存储完整模型
os.makedirs('tmp/pytorch', exist_ok=True)
torch.save(model, 'tmp/pytorch/model.pth')
print('Saved PyTorch Model to tmp/pytorch/model.pth')
model = torch.load('tmp/pytorch/model.pth')

Saved PyTorch Model to tmp/pytorch/model.pth


In [ ]:
# 模型预测
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


## Text

### Transformer
1. embedding layer
2. positional encoding: 统计单词顺序？
3. TransformerEncoderLayer
4. square attention mask: 遮盖未来单词

In [ ]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import functional as F, TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset


class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    
    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


class TransformerModel(nn.Module):
    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.encoder = nn.Embedding(ntoken, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()
        
    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)


In [ ]:
torch.arange(2).unsqueeze(1) * torch.ones(3)

tensor([[0., 0., 0.],
        [1., 1., 1.]])

In [ ]:
torch.arange(2).unsqueeze(1).unsqueeze(1) * torch.ones(3)

tensor([[[0., 0., 0.]],

        [[1., 1., 1.]]])

In [ ]:
t1 + torch.arange(2).unsqueeze(1) * torch.ones(3)

tensor([[[0., 0., 0.],
         [1., 1., 1.]],

        [[0., 0., 0.],
         [1., 1., 1.]]])

In [ ]:
import torch
t1 = torch.zeros(2, 2, 3)
t1 + torch.arange(2).unsqueeze(1).unsqueeze(1) * torch.ones(3)

tensor([[[0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.]]])

In [ ]:
tmp = nn.Embedding(6, 6).weight.data
tmp.uniform_?

In [ ]:
torch.sin(torch.arange(10).unsqueeze(1)) * torch.exp(torch.arange(0, 10, 2))

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  6.2177e+00,  4.5943e+01,  3.3947e+02,  2.5084e+03],
        [ 9.0930e-01,  6.7188e+00,  4.9646e+01,  3.6684e+02,  2.7106e+03],
        [ 1.4112e-01,  1.0427e+00,  7.7049e+00,  5.6932e+01,  4.2067e+02],
        [-7.5680e-01, -5.5921e+00, -4.1320e+01, -3.0532e+02, -2.2560e+03],
        [-9.5892e-01, -7.0855e+00, -5.2355e+01, -3.8686e+02, -2.8585e+03],
        [-2.7942e-01, -2.0646e+00, -1.5256e+01, -1.1272e+02, -8.3293e+02],
        [ 6.5699e-01,  4.8545e+00,  3.5870e+01,  2.6505e+02,  1.9584e+03],
        [ 9.8936e-01,  7.3104e+00,  5.4017e+01,  3.9914e+02,  2.9492e+03],
        [ 4.1212e-01,  3.0452e+00,  2.2501e+01,  1.6626e+02,  1.2285e+03]])

In [ ]:
torch.empty?